In [1]:
import pandas as pd
import feather

In [2]:
class bagOfVisits:    
    
    #to get the device input
    def device_selection(device_inp):
        
        if device_inp == 1:
            device = "desktop"
            
        elif device_inp == 2:
            device = "mapp"
            
        elif device_inp == 3:
            device = "mew"
        
        elif device_inp == 4:
            device = "tablet"
        else:
            print("Error!! Incorrect device input :")
            return bagOfVisits.device_selection(int(input("Select the device:\n 1 for Desktop \n 2 for Mobile App\n 3 for Mobile Web\n 4 for Tablet: ")))
              
        return device    
       

    #Calcualtor to calculate previous month number
    def prev_month_calculator(month):
       
        if month == 12:
            prev_month = 0
            
        elif month == 1:
            prev_month = 12
        
            
        elif month > 1 and month < 12:
            prev_month = month-1
            
        else:
            print("Error!! Incorrect month input :")
            return bagOfVisits.prev_month_calculator
            
        return prev_month
    
    #Calcualtor to calculate next month number
    def next_month_calculator(month):
        
        if month == 12:
            next_month = 1
            
        elif month == 9:
            next_month = 0
    
            
        elif month >= 1 and month < 12:
            next_month = month+1
            
        else:
            print("Error!! Incorrect month input :")
            
        return next_month
    
    
    #To get all the required months
    def month(mapping_table,input_month):
        
        final_mapping_table = mapping_table
        
        final_mapping_table['Previous Month Number'] = final_mapping_table['Current Month Number'].apply(\
                                                                                                         bagOfVisits.prev_month_calculator)
        
        final_mapping_table['Next Month Number'] = final_mapping_table['Current Month Number'].apply(\
                                                                                                     bagOfVisits.next_month_calculator)
        
        final_mapping_table_1 = pd.merge(final_mapping_table, mapping_table, left_on = 'Previous Month Number',\
                                         right_on = 'Current Month Number', how = 'left')
        
        final_mapping_table_1.drop(columns = ['Current Month Number_y', 'Previous Month Number_y', 'Next Month Number_y'],\
                                   inplace = True)
        
        final_mapping_table_1.columns = ['Current Month Number', 'Current Month Name', 'Previous Month Number',\
                                         'Next Month Number', 'Previous Month Name']
        
        final_mapping_table_2 = pd.merge(final_mapping_table_1, mapping_table, left_on = 'Next Month Number',\
                                         right_on = 'Current Month Number', how = 'left')
        
        
        final_mapping_table_2.drop(columns = ['Current Month Number_y', 'Previous Month Number_y', \
                                      'Next Month Number_y'], inplace = True)

        
        final_mapping_table_2.columns = ['Current Month Number', 'Current Month Name', 'Previous Month Number', \
                                 'Next Month Number', 'Previous Month Name', 'Next Month Name']
        
        final_mapping_table_2.fillna("Data Not Available", inplace = True) 
        
        
        prev_month = int(final_mapping_table_2['Previous Month Number'].loc[final_mapping_table_2\
                                                                            ['Current Month Name'] == input_month])
        
        
        next_month = int(final_mapping_table_2['Next Month Number'].loc[final_mapping_table_2\
                                                                        ['Current Month Name'] == input_month])
        
        
        current_month = int(final_mapping_table_2['Current Month Number'].loc[final_mapping_table_2\
                                                                              ['Current Month Name'] == input_month])
        
        prev_month_name = final_mapping_table_2['Previous Month Name'].loc\
        [final_mapping_table_2['Current Month Name'] == input_month].reset_index().loc[0]['Previous Month Name']
        
        
        next_month_name = final_mapping_table_2['Next Month Name'].loc\
                              [final_mapping_table_2['Current Month Name'] == input_month].reset_index().loc[0]['Next Month Name']
        
        
        return prev_month, current_month, next_month,prev_month_name,input_month,next_month_name,final_mapping_table_2
    
    
    def average_sd(input_month,df,device_op,page):
        var = df.loc[(df['Page'] == page) & (df['Device'] == device_op) & (df['Input_month'] == input_month)] 
        total_session = var['Session_count'].iloc[0]
        Avg = var['Avg'].iloc[0]
        std = var['Std'].iloc[0]
        days = var['Number_of_days'].iloc[0] 
        return total_session,Avg,std,days      

       
   

    def func_call(device_inp,landingPage,month_name,metric):
        device_op = bagOfVisits.device_selection(device_inp)
        xlsx = pd.ExcelFile('non-proportional-prepowerd-data.xlsx')
        if metric == 1:
            df = pd.read_excel(xlsx, 'RPS')
        elif metric == 2:
            df = pd.read_excel(xlsx, 'AOV')
        elif metric == 3:
            df = pd.read_excel(xlsx, 'RPV')
        elif metric == 4:
            df = pd.read_excel(xlsx, 'UPO')
        mapping_table = pd.read_csv('month_mapping.csv')
        prev_month, current_month, next_month,a,b,c,d = bagOfVisits.month(mapping_table,month_name)
        total_session,Avg,std,days = bagOfVisits.average_sd(month_name,df,device_op,landingPage)
        return total_session,Avg,std,days,prev_month, current_month, next_month,a,b,c,d

        